In [4]:
pip install pyspark

    8% |██▉                             | 25.1MB 120kB/s eta 0:35:32^C  2% |▊                               | 6.3MB 76kB/s eta 1:00:00    2% |█                               | 8.4MB 59kB/s eta 1:17:03    3% |█▎                              | 11.0MB 73kB/s eta 1:01:16    4% |█▋                              | 13.8MB 143kB/s eta 0:31:05    5% |█▋                              | 14.2MB 210kB/s eta 0:21:12    7% |██▍                             | 21.5MB 91kB/s eta 0:47:29

Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [7]:
# Intialization
import os
import gc
import sys
import shutil

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'


In [8]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = SparkSession.builder.appName("Final Project").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [9]:
#Click Here
#Uname and UIds
df_live_user_name=spark.read.format('parquet').load('file:///home/talentum/Project/ReviewDf_name/')
#Restro Table
df_only_restors=spark.read.format('parquet').load('file:///home/talentum/FinalTable/Parq/FBusinessPar.parquet')
#User Table
df_only_users=spark.read.format('parquet').load('file:///home/talentum/FinalTable/Parq/fuser.parquet')

In [10]:
#Actual Recomendations 
def get_top_n_recommendations(model,user_id, n=50):
    recommendations = model.recommendProducts(user_id, n)
    model = None  # Dereference the model
    gc.collect()  # Force garbage collection to free up memory
    
    # Convert the recommendations to a list of Rows
    rows = [Row(user_id=user_id, itemId_numeric=r.product, rating=r.rating) for r in recommendations]
    
    # Create a Spark DataFrame from the list of Rows
    recommendations_df = spark.createDataFrame(rows)
    
    return recommendations_df


In [13]:
#for filters and sorts 
def recomend_restros(model,user_id_numeric,u_state='',u_category='',u_nr=5):
    # Get the top N recommendations for the user
    recommendations_df = get_top_n_recommendations(model,user_id_numeric, n=50)

    # Filter the main DataFrame using the recommendations
    filtered_df = df_only_restors \
        .filter((col('state').rlike(f"(?i).*{u_state}.*"))& 
            (col('categories').rlike(f"(?i).*{u_category}.*")))

    # Join the recommendations with the filtered DataFrame
    result_df = filtered_df.join(recommendations_df, on='itemId_numeric', how='inner')

    # Collect the top 5 results based on the rating
    top_5_results = result_df.orderBy(col("rating").desc()).limit(u_nr)

    # Show the top 5 recommendations
    top_5_results.select('itemId_numeric','name','address', 'city',
     'state','categories','stars').show()

In [14]:
def new_user(userId,uname):
    from pyspark.sql.types import DoubleType, IntegerType, StructType, StructField,StringType
    schema = StructType([
    StructField("userId_numeric", StringType(), True),
    StructField("name", StringType(), True)
    ])
    data = [(userId,uname)]
    new_user_df = spark.createDataFrame(data, schema)
    new_user_df.write.mode("append").parquet("file:///home/talentum/Project/ReviewDf_name/")
    print("done")

In [15]:
def lets_rate(user_Id):
    from pyspark.sql.types import DoubleType, IntegerType, StructType, StructField
    #df_numeric=spark.read.format('parquet').load("file:///home/talentum/Project/ReviewDf/")
    #df_temp=df_numeric
    #-
    restro_Id=int(input("Select Restorant you Like (ID):"))
    
    
    df_r=df_only_restors.where(col('itemId_numeric')==int(restro_Id))
    restro_name = df_r.select('name').collect()[0][0] 
    #-
    u_stars=float(input(f"Please Rate {restro_name}(out of 5):"))
    #Updating the df
    
    schema = StructType([
    StructField("userId_numeric", IntegerType(), True),
    StructField("itemId_numeric", IntegerType(), True),
    StructField("stars", DoubleType(), True)
    ])
    data = [(user_Id,restro_Id , u_stars)]
    df_to_update=spark.createDataFrame(data,schema)

    df_to_update.write.mode("append").parquet("file:///home/talentum/Project/ReviewDf/")
    
    print("Thanks For the rating... ")
    retrain_model()
    print("---Model_Saved---")


    

In [16]:
def retrain_model():
    from pyspark.mllib.recommendation import ALS, Rating
    from pyspark.sql.functions import col
    df_live=spark.read.format('parquet').load("file:///home/talentum/Project/ReviewDf/")
    ratings = df_live.rdd.map(lambda row: Rating(row["userId_numeric"], row["itemId_numeric"], row["stars"]))
     
    model_path = 'file:///home/talentum/Project/Model1'
    
    if os.path.exists(model_path):
    # Delete the existing model directory
        shutil.rmtree(model_path)
    
    
    # Train ALS model
    rank = 10
    numIterations = 10
    block_size=-1 # parallel computing
    model1 = ALS.train(ratings, rank, numIterations,blocks=block_size)
    print("---Model_Trained---")
    #rf_model.write().overwrite().save(rf_model_path)
    model1.save(spark.sparkContext, model_path)
    

In [17]:
def top_5():
    print("---------------------Top Rated Places ----------------------")
    top_5_results = df_only_restors.orderBy(col("stars").desc()).select('itemId_numeric','name','address', 'city',
    'state','categories','stars').limit(5)
    top_5_results.show()

In [18]:
# Function to get top 50 recomendation RETURNS A DF OBJECT
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import DoubleType, IntegerType, StructType, StructField
#og user 234659
user_Id=input("Enter your User  Id (existing user)/ alphabate for new user :")
if(user_Id.isdigit() and  df_live_user_name.filter(col('userId_numeric').isin([int(user_Id)])).count() > 0):
    
    
    path ='file:///home/talentum/Project/Model'

    from pyspark.mllib.recommendation import MatrixFactorizationModel
    model = MatrixFactorizationModel.load(sc, path)
    # Get recommendations from the model
    
    df_u_all=spark.read.format('parquet').load("file:///home/talentum/Project/ReviewDf_name/")
    df_user=df_u_all.where(col('userId_numeric')==int(user_Id))
    user_name = df_user.select("name").collect()[0][0] 
    user_Id=int(user_Id)
    print(f"Welcome Back {user_name} !")
    
    
    top_5()
    
    
    print("---------------------Top 5 Recommendations for you---------------------")
    recomend_restros(model,user_id_numeric=user_Id)
    lets_rate(user_Id)
    
    print("-----------------------------------------------------------------------")
    
    path = 'file:///home/talentum/Project/Model1'
    
    from pyspark.mllib.recommendation import MatrixFactorizationModel
    model1 = MatrixFactorizationModel.load(sc, path)
    print("We  Are Ready...")
    #ustate=input("Enter Your state(0  for none ):")
    #if(ustate=='0'):
    #    ustate=''
    print("---------------------Top 5 Recommendations for you---------------------")
    recomend_restros(model1,user_id_numeric=user_Id)#,u_state=ustate)
    
    
    
else :
    df_numeric=spark.read.format('parquet').load("file:///home/talentum/Project/ReviewDf/")
    print("---------------------Id Generated for you---------------------")
    #-
    max_Id = df_numeric.agg({"userId_numeric": "max"}).collect()[0]
    new_Id=max_Id["max(userId_numeric)"]+1
    print(f"Your Id is :{new_Id}")
    uname=input("Enter Your name:")
    new_user(new_Id,uname)
    
    top_5()
    
    lets_rate(int(new_Id))
    
    print("-----------------------------------------------------------------------")
    
    path = 'file:///home/talentum/Project/Model1'
    
    from pyspark.mllib.recommendation import MatrixFactorizationModel
    model1 = MatrixFactorizationModel.load(sc, path)
    
    print("---------------------Top 5 Recommendations for you---------------------")
    recomend_restros(model1,user_id_numeric=new_Id)
    
    

Enter your User  Id (existing user)/ alphabate for new user :77
Welcome Back Eliot !
---------------------Top Rated Places ----------------------
+--------------+--------------------+-----------------+------------+-----+--------------------+-----+
|itemId_numeric|                name|          address|        city|state|          categories|stars|
+--------------+--------------------+-----------------+------------+-----+--------------------+-----+
|         36081|    The Mad Griddle |   2127 E 10th St|Indianapolis|   IN|Arts & Entertainm...|  5.0|
|         64655|          The Office|     248 W 1st St|        Reno|   NV|Nightlife, Lounge...|  5.0|
|           191|     The Pepper Pott| 4611 Alabama Ave|   Nashville|   TN|Caribbean, Food, ...|  5.0|
|         47614|       Mr. Margarita|    Truck Inn Way|     Fernley|   NV|   Food Trucks, Food|  5.0|
|         58054|The Music Box Vil...|4557 N Rampart St| New Orleans|   LA|Local Flavor, Pub...|  5.0|
+--------------+--------------------+-

In [131]:
df_u_all=spark.read.format('parquet').load("file:///home/talentum/Project/ReviewDf_name/")
df_u_all.where(col('userId_numeric')==234660).show()

+--------------+----+
|userId_numeric|name|
+--------------+----+
+--------------+----+



In [ ]:
def recommentation():
    